# Building a Custom Agent with Flotorch and LangChain

This notebook demonstrates how to build and run a simple AI agent using the Flotorch platform and the LangChain framework. The agent will be equipped with custom tools to perform specific functions.

### Key Objectives:
- Install the required Flotorch package.
- Define custom tools for the agent.
- Configure an agent using a prompt template and bind it to the tools.
- Utilize the `FlotorchLangChainLLM` to power the agent's reasoning capabilities.
- Assemble and run the agent workflow using an `AgentExecutor`.

## 1. Environment Setup and Imports

The following cells handle the complete setup of the environment. This includes:
1. **Installing** the necessary Flotorch SDK package.
2. **Importing** essential libraries.
3. **Configuring** the API credentials and model endpoint.

In [ ]:
# install flotorch langchain package
%pip install  flotorch[langchain]

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "https://qa-gateway.flotorch.cloud"
FLOTORCH_MODEL = "<flotorch model>"

In [ ]:
# Import LangChain and related modules
from flotorch.langchain.llm import FlotorchLangChainLLM
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool

print("Imported necessary libraries successfully")

## 2. Defining a Custom Tool

To extend the agent's capabilities, we define custom tools. The `@tool` decorator from LangChain transforms a standard Python function into a tool that the agent can utilize. This example creates an `analyze_text` tool, which takes a string as input and returns a statistical analysis. The function's docstring serves as its description, helping the agent understand how and when to use it.

In [ ]:
@tool
def analyze_text(text: str) -> str:
    """
    Analyze text and provide comprehensive statistics.
    
    Args:
        text (str): Text to analyze
        
    Returns:
        str: Formatted text analysis with statistics
    """
    word_count = len(text.split())
    char_count = len(text)
    char_count_no_spaces = len(text.replace(' ', ''))
    sentences = text.count('.') + text.count('!') + text.count('?')
    
    return f"""Text Analysis:
- Words: {word_count}
- Characters (with spaces): {char_count}
- Characters (without spaces): {char_count_no_spaces}
- Sentences: {sentences}"""


@tool
def weather(city: str) -> str:
    """Get weather for a city. Available cities: New York, London, Tokyo, Paris."""
    city = city.lower().strip()

    weather_info = {
        "new york": "Sunny, 72°F, Humidity: 45%",
        "london": "Cloudy, 15°C, Humidity: 70%",
        "tokyo": "Rainy, 25°C, Humidity: 80%",
        "paris": "Partly cloudy, 18°C, Humidity: 60%"
    }

    if city in weather_info:
        return f"Weather in {city.title()}: {weather_info[city]}"
    else:
        return f"I don't have weather data for '{city}'. Try: New York, London, Tokyo, or Paris."

tools = [analyze_text, weather]

print("Custom tool defined successfully.")

## 3. Configuring the Language Model

The agent requires a Large Language Model (LLM) to act as its brain. We instantiate `FlotorchLangChainLLM`, passing our API credentials and the desired model ID. This object serves as the bridge between our LangChain agent and the powerful reasoning capabilities of the Flotorch platform.

In [ ]:
model  =  FlotorchLangChainLLM(
        model_id=FLOTORCH_MODEL,
        api_key=FLOTORCH_API_KEY,
        base_url=FLOTORCH_BASE_URL,
    )
print("FlotorchLLM initialized")

## 4. Creating the Agent

With the tool and LLM configured, we now define the agent's core logic. This involves two key LangChain components:

- **Prompt Template**: We use `ChatPromptTemplate` to structure the instructions given to the LLM. This template defines the agent's persona (e.g., 'a helpful assistant'), includes placeholders for user input (`{input}`) and intermediate steps (`agent_scratchpad`), and guides the agent's behavior.
- **Agent**: We use the `create_openai_functions_agent` function to bind the LLM, the tools, and the prompt together. This function creates the agent logic that can decide which tool to use based on the user's input.

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant with access to tools for calculations, weather, and information search.
    Use the appropriate tool when needed, or answer directly if you can.
    Keep your responses clear and concise."""),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])


agent = create_openai_functions_agent(model,tools,prompt)

print(f"Agent is created successfully")

## 5. Running the Agent with an Executor

The final step is to create a runtime environment for our agent. In LangChain, this is handled by the `AgentExecutor`. The `AgentExecutor` takes the agent and the tools as input and is responsible for executing the agent's decisions. It calls the agent, determines which tool to use, executes that tool, and passes the result back to the agent to formulate a final response. We start the process by calling the `invoke()` method with the user's input.

In [ ]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True
)

user_input = input("user: ")
print(f"User: {user_input}")
response = agent_executor.invoke({"input":user_input})
print(f"Assistant: {response['output']}")


## Summary

This notebook demonstrated the end-to-end process of creating a functional AI agent using the Flotorch ADK and LangChain. 

The key components included:

1. **Custom Tool Implementation**: A Python function was seamlessly converted into a usable tool for the agent using LangChain's `@tool` decorator.
2. **Model Configuration**: The `FlotorchLangChainLLM` was configured to provide the agent with its reasoning ability.
3. **Agent Creation**: A prompt template (`ChatPromptTemplate`) and an agent constructor (`create_openai_functions_agent`) were used to define the agent's behavior and access to tools.
4. **Agent Execution**: The `AgentExecutor` was used to run the agent, orchestrating the interaction between the LLM and the custom tools to produce the final output.
    
This example showcases the power and flexibility of the Flotorch platform for building customized, tool-augmented AI agents with LangChain.
   
  